## 410 Final Project: Generating Summaries for News Articles
Aaron Kuhstoss, Shalin Mehta, and Aleksandra Grigortsuk

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from langdetect import detect


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


 ### Data preprocessing
 

In [2]:
# import the dataset
df = pd.read_csv("Latest_News.csv")

# filtering dataset with English articles and non-NA 
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

# Apply the language detection function to df
df['detected_language'] = df['content'].apply(detect_language)
english_articles = df[df['detected_language'] == 'en']

# every article has content and title


### Pipeline Construction
1. Summarization pipeline
2. Categorization pipline (optional)
*does not need to be fully implemented by 11/1 milestone*

0

2234.840919199407